# 67373视频推荐系统

In [1]:
from sklearn.model_selection import train_test_split # pip install scikit-learn
import numpy as np # pip install numpy

In [2]:
# 从本地读取数据集
f = open("dataset/dataset.txt", "r")
lines = f.read().splitlines()
dataset = []
for line in lines:
    # arr格式为: [title, cate_name, cate_id, duration, creation_time, video_id]
    arr = line.split(',')
    dataset.append(arr)

## 创建codebook

In [3]:
# 提取数据集每行的前两个元素（即title和cate_name），以字为单位创建codebook
codebook = set()
for data in dataset:
    codebook.update(data[0], data[1])
codebook = list(codebook)
print('codebook长度: {}个字'.format(len(codebook)))
print('codebook内容:\n{}'.format(codebook))

codebook长度: 671个字
codebook内容:
['时', '曲', '再', '海', '财', '，', '诈', '由', '纯', '诱', '别', '-', '发', '旅', '春', '翅', 'T', '是', '区', '开', '第', '每', '吗', '络', '太', '行', '曹', '常', '娱', '友', '谢', '让', '4', 'い', '王', '画', '乎', '医', '啊', '思', '大', '教', '等', '泡', '何', '阴', '冷', '故', 'て', '两', '度', '位', '孤', '吹', '猫', '哭', '窍', '盼', '罗', '暴', '还', '红', '膀', '但', '白', '唱', '段', '馆', '的', '经', '！', '总', '乘', '些', '～', '轮', '雪', '考', '视', '遥', 'U', '゙', '昭', '少', '新', '什', '三', '刻', '喝', '迷', '烧', 'W', '封', '人', '胎', '育', '病', '作', '、', '月', '金', '点', '贝', '方', '（', '满', '认', '快', '给', '尘', '理', '冬', '解', 'j', '陌', '诗', '闷', '楼', '交', '者', '核', '东', '《', '饭', '狗', '波', '次', '条', '起', '球', '幸', '号', '）', '丁', '散', '赚', '路', 'O', '己', '现', '动', '家', '目', '难', '界', '桥', '养', '县', '醉', 'r', '忘', '子', '0', '炸', '硬', '戏', '下', '另', '相', '愁', '到', '朋', '婆', '鲁', '傻', 'b', '见', '岛', '笑', '写', '首', '话', '能', '黑', '良', '隐', '文', 't', '1', '秋', '要', '寞', 'I', '会', '米', '享', '妙', '广', '钟', '着', '影', '汉', '坐', '怀',

## Bag of Words算法

In [4]:
# 参数inputs可以是字符串也可以是字符串数组
def BoW(inputs, codebook):
    # 初始化返回结果
    # vec是该inputs的one-hot向量表示
    # indices是该inputs在codebook里的索引
    vec = [0] * len(codebook)
    indices = []
    # 处理inputs，如果inputs是数组，则拼接成一个字符串
    # 例如：inputs是["童话镇", "视频-直播回放"]，则拼接成 "童话镇视频-直播回放"
    if (type(inputs) == list):
        inputs = ''.join(inputs)
    # 匹配codebook
    try:
        for char in list(inputs):
            index = codebook.index(char)
            vec[index] = 1
            indices.append(index)
    except:
        print('ValueError: value is not in list')
    return np.array(vec), np.array(indices)

In [5]:
# 测试
# vec, indices = BoW("童话镇", codebook)
# vec, indices = BoW(["童话镇"], codebook)
vec, indices = BoW(["童话镇", "视频-直播回放"], codebook)
print('向量长度为:\n{}'.format(len(vec)))
print('向量表示为:\n{}'.format(vec))
print('索引位置为:\n{}'.format(indices))

向量长度为:
671
向量表示为:
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

## 创建训练集和测试集

In [6]:
# 首先把数据集中的字符串数据（标题，分类）通过BoW算法用向量表示
x = [] # 用来训练的输入向量
y = [] # 判断训练结果的标签
for data in dataset:
    title = data[0]
    category = data[1]
    vec, indices = BoW([title, category], codebook)
    x.append(vec)
    y.append(title)
x = np.array(x)
y = np.array(y)
print('x的纬度: {}'.format(x.shape))
print('y的纬度: {}'.format(y.shape))

x的纬度: (312, 671)
y的纬度: (312,)


In [7]:
# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(x, y)
print('训练集x的纬度: {}'.format(x_train.shape))
print('训练集y的纬度: {}'.format(y_train.shape))
print('测试集x的纬度: {}'.format(x_test.shape))
print('测试集y的纬度: {}'.format(y_test.shape))

训练集x的纬度: (234, 671)
训练集y的纬度: (234,)
测试集x的纬度: (78, 671)
测试集y的纬度: (78,)
